In [1]:
# !pip install flask

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from PIL import Image
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
import io
# from flask import Flask, render_template, request, Markup
from model import ResNet9
from disease import disease_dic
from fertilizer import fertilizer_dic

C:\Users\91941\AppData\Local\Temp\ipykernel_5216\4189171892.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### **Dictionary for disease namme categories**

In [3]:
disease_classes = ['Apple___Apple_scab',
                   'Apple___Black_rot',
                   'Apple___Cedar_apple_rust',
                   'Apple___healthy',
                   'Blueberry___healthy',
                   'Cherry_(including_sour)___Powdery_mildew',
                   'Cherry_(including_sour)___healthy',
                   'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
                   'Corn_(maize)___Common_rust_',
                   'Corn_(maize)___Northern_Leaf_Blight',
                   'Corn_(maize)___healthy',
                   'Grape___Black_rot',
                   'Grape___Esca_(Black_Measles)',
                   'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
                   'Grape___healthy',
                   'Orange___Haunglongbing_(Citrus_greening)',
                   'Peach___Bacterial_spot',
                   'Peach___healthy',
                   'Pepper,_bell___Bacterial_spot',
                   'Pepper,_bell___healthy',
                   'Potato___Early_blight',
                   'Potato___Late_blight',
                   'Potato___healthy',
                   'Raspberry___healthy',
                   'Soybean___healthy',
                   'Squash___Powdery_mildew',
                   'Strawberry___Leaf_scorch',
                   'Strawberry___healthy',
                   'Tomato___Bacterial_spot',
                   'Tomato___Early_blight',
                   'Tomato___Late_blight',
                   'Tomato___Leaf_Mold',
                   'Tomato___Septoria_leaf_spot',
                   'Tomato___Spider_mites Two-spotted_spider_mite',
                   'Tomato___Target_Spot',
                   'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
                   'Tomato___Tomato_mosaic_virus',
                   'Tomato___healthy']

disease_model_path = 'plant_disease_model.pth'
disease_model = ResNet9(3, len(disease_classes))
disease_model.load_state_dict(torch.load(
    disease_model_path, map_location=torch.device('cpu')))
disease_model.eval()

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

### *Disease model*

### Following function is to predict disease of plant from image

In [4]:
def predict_image(img, model=disease_model):
    """
    Transforms image to tensor and predicts disease label
    :params: image
    :return: prediction (string)
    """
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
    image = Image.open(img)
    img_t = transform(image)
    img_u = torch.unsqueeze(img_t, 0)

    # Get predictions from model
    yb = model(img_u)
    # Pick index with highest probability
    _, preds = torch.max(yb, dim=1)
    prediction = disease_classes[preds[0].item()]
    # Retrieve the class label
    return prediction

In [5]:
# img = "output.JPG"
img = 'C:\\Users\\91941\\OneDrive\\Desktop\\iot_proj\\leaf.jpg'
prediction = predict_image(img, disease_model)
prediction = str(disease_dic[prediction])

In [6]:
print(prediction)

 <b>Crop</b>: Apple <br/>Disease: Apple Scab<br/>
        <br/> Cause of disease:

        <br/><br/> 1. Apple scab overwinters primarily in fallen leaves and in the soil. Disease development is favored by wet, cool weather that generally occurs in spring and early summer.

        <br/> 2. Fungal spores are carried by wind, rain or splashing water from the ground to flowers, leaves or fruit. During damp or rainy periods, newly opening apple leaves are extremely susceptible to infection. The longer the leaves remain wet, the more severe the infection will be. Apple scab spreads rapidly between 55-75 degrees Fahrenheit.
        <br/><br/> How to prevent/cure the disease <br/>
        <br/>1. Choose resistant varieties when possible.

        <br/>2. Rake under trees and destroy infected leaves to reduce the number of fungal spores available to start the disease cycle over again next spring
        
        <br/>3. Water in the evening or early morning hours (avoid overhead irrigation) t

### Follwing function will take soil content and will measure balanced or not and will give the way to get rid if any unbalanced N, P, K is present.

In [7]:
def fert_recommend(crop_name, N, P, K):

    df = pd.read_csv('fertilizer.csv')

    nr = df[df['Crop'] == crop_name]['N'].iloc[0]
    pr = df[df['Crop'] == crop_name]['P'].iloc[0]
    kr = df[df['Crop'] == crop_name]['K'].iloc[0]

    n = nr - N
    p = pr - P
    k = kr - K
    temp = {abs(n): "N", abs(p): "P", abs(k): "K"}
    max_value = temp[max(temp.keys())]
    if max_value == "N":
        if n < 0:
            key = 'NHigh'
        else:
            key = "Nlow"
    elif max_value == "P":
        if p < 0:
            key = 'PHigh'
        else:
            key = "Plow"
    else:
        if k < 0:
            key = 'KHigh'
        else:
            key = "Klow"

    response = str(fertilizer_dic[key])

    return response

In [8]:
# Open the text file in read mode
path = 'C:\\Users\\91941\\OneDrive\\Desktop\\iot_proj\\composition.txt'

with open(path, 'r') as file:
    data = file.readline().strip()
    components = data.split(',')
    N, P, K = None, None, None
    print(components)
    for component in components:
        key, value = component.split(':')
        if key == 'N':
            N = int(value)
        elif key == 'P':
            P = int(value)
        elif key == 'K':
            K = int(value)
crop = 'apple'
response = fert_recommend(crop, N, P, K)

['N:80', 'P:40', 'K:40']


In [9]:
print(response)
response = 'crop name : <b>'+ crop +'</b><br>' + response

The K value of your soil is low.
        <br/>Please consider the following suggestions:

        <br/><br/>1. Mix in muricate of potash or sulphate of potash
        <br/>2. Try kelp meal or seaweed
        <br/>3. Try Sul-Po-Mag
        <br/>4. Bury banana peels an inch below the soils surface
        <br/>5. Use Potash fertilizers since they contain high values potassium
        


In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
def mailing_func(body,subject,image_path):
    sender_email = ''
    sender_password = ""
    receiver_email_list = ['anant23k.kiran@gmail.com','shubhiagarwal2494@gmail.com','harshiitg2025@gmail.com','arghyadip@iitg.ac.in']
    # receiver_email_list = ['anant23k.kiran@gmail.com','shubhiagarwal2494@gmail.com','harshiitg2025@gmail.com']
    # receiver_email_list = ['harshiitg2025@gmail.com']
    for receiver_email in receiver_email_list:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html'))
        if(len(image_path)>0):
            with open(image_path, 'rb') as img_file:
                # Create an image MIMEImage object
                img = MIMEImage(img_file.read())
                # Define the image ID
                img.add_header('Content-ID', '<image1>')
                # Attach the image to the email
                msg.attach(img)

        try:
            smtp_server = smtplib.SMTP('smtp.gmail.com', 587)

            smtp_server.starttls()
            smtp_server.login(sender_email, sender_password)
            smtp_server.sendmail(sender_email, receiver_email, msg.as_string())
            smtp_server.quit()

            print("Email sent successfulGmail ly!")
        except Exception as e:
            print("An error occurred while sending the email:", str(e))
    return 


### Exporting Results

In [15]:


subject = 'Soil update for ' + crop
body = response
mailing_func(body,subject,'')


Email sent successfulGmail ly!
Email sent successfulGmail ly!
Email sent successfulGmail ly!


In [16]:
file_path_image = "soil_output.txt"
with open(file_path_image, "w") as file:
    file.write(body)

In [17]:
subject = 'health update for ' + crop
body = prediction
mailing_func(body,subject,img)

Email sent successfulGmail ly!
Email sent successfulGmail ly!
Email sent successfulGmail ly!


In [18]:
file_path_image = "leaf_output.txt"
with open(file_path_image, "w") as file:
    file.write(body)